In [ ]:
import numpy as np
from glob import glob
import os
import mne 
import pandas as pd
import matplotlib.pyplot as plt
from dotenv import main

import datetime

In [ ]:
from sklearn.model_selection import GroupKFold,LeaveOneGroupOut
from sklearn.preprocessing import StandardScaler
from sklearn.metrics import confusion_matrix, classification_report
import seaborn as sns
import random

In [ ]:
import tensorflow as tf

from tensorflow.keras.models import Sequential
from tensorflow.keras.models import Model

from tensorflow.keras.layers import Input, GRU, Dense, Conv1D

from tensorflow.keras.layers import BatchNormalization, LeakyReLU, MaxPool1D,\
GlobalAveragePooling1D, Dropout, AveragePooling1D, Flatten, Concatenate


from tensorflow.keras.optimizers import Adam
from tensorflow.keras.callbacks import ReduceLROnPlateau, TensorBoard

from tensorflow.keras import backend as K

In [ ]:
print("Num GPUs Available: ", len(tf.config.list_physical_devices('GPU')))

In [ ]:
print(tf.config.list_physical_devices('GPU'))

In [ ]:
import matplotlib.pyplot as plt
import seaborn as sns

def plot_training_results(training_losses, training_accuracies, validation_losses, validation_accuracies, epochs=20, save_plots=False, save_dir=None, leg=True):
    fig, axs = plt.subplots(2, 2, figsize=(15, 10), sharex=True)  # Sharing x-axis for all subplots
    
    # Settings for aesthetics
    marker_style = dict(linestyle='-', marker='o', markersize=5)
    
    # Plotting training loss
    for fold, losses in enumerate(training_losses):
        axs[0, 0].plot(range(1, epochs+1), losses, label=f'Fold {fold+1}', **marker_style)
    axs[0, 0].set_title('Training Loss')
    axs[0, 0].set_xlabel('Epoch')
    axs[0, 0].set_ylabel('Loss')
    if leg:
        axs[0, 0].legend()
        
    axs[0, 0].grid(True)

    # Plotting training accuracy
    for fold, accuracies in enumerate(training_accuracies):
        axs[0, 1].plot(range(1, epochs+1), accuracies, label=f'Fold {fold+1}', **marker_style)
    axs[0, 1].set_title('Training Accuracy')
    axs[0, 1].set_xlabel('Epoch')
    axs[0, 1].set_ylabel('Accuracy')
    if leg:
        axs[0, 1].legend()
    axs[0, 1].grid(True)

    # Plotting validation loss
    for fold, losses in enumerate(validation_losses):
        axs[1, 0].plot(range(1, epochs+1), losses, label=f'Fold {fold+1}', **marker_style)
    axs[1, 0].set_title('Validation Loss')
    axs[1, 0].set_xlabel('Epoch')
    axs[1, 0].set_ylabel('Loss')

    if leg:     
        axs[1, 0].legend()
    axs[1, 0].grid(True)

    # Plotting validation accuracy
    for fold, accuracies in enumerate(validation_accuracies):
        axs[1, 1].plot(range(1, epochs+1), accuracies, label=f'Fold {fold+1}', **marker_style)
    axs[1, 1].set_title('Validation Accuracy')
    axs[1, 1].set_xlabel('Epoch')
    axs[1, 1].set_ylabel('Accuracy')
    if leg:
        axs[1, 1].legend()
    axs[1, 1].grid(True)

    plt.tight_layout()
    
    if save_plots:
        if save_dir:
            # Ensure directory exists
            os.makedirs(save_dir, exist_ok=True)
            file_path = os.path.join(save_dir, 'training_validation_metrics.png')
        else:
            file_path = 'training_validation_metrics.png'
        
        plt.savefig(file_path)  # Saves the figure to a file
        print(f"Saved plots to '{file_path}'")
    
    plt.show()

In [ ]:
def read_data(file_path):
    data = mne.io.read_raw_edf(file_path, preload=True) #preload is faster but consumes more memory
    data.set_eeg_reference()
    data.filter(l_freq=0.5, h_freq=45) #usual band for eeg 
    epochs = mne.make_fixed_length_epochs(data, duration=5,overlap=1)
    array = epochs.get_data()

    return array

In [ ]:
def shuffle_group_labels(groups):
    # Ensure the input groups are integers and extract the unique groups
    unique_groups = np.unique(groups)
    
    # Shuffle the unique group labels
    shuffled_groups = np.random.permutation(unique_groups)
    
    # Create a mapping from original group labels to shuffled labels
    group_mapping = dict(zip(unique_groups, shuffled_groups))
    
    # Remap the original group labels to shuffled labels
    shuffled_group_labels = np.vectorize(group_mapping.get)(groups)
    
    return shuffled_group_labels

## Read the Data

In [ ]:
# Load the environment variables from the .env file
main.load_dotenv()

# Access the variable
dataset_path = os.getenv('DATASET_PATH')


In [ ]:
files_paths = glob(dataset_path + f"/*.edf")

In [ ]:
print(len(files_paths))

In [ ]:
healthy_files_paths = [i for i in files_paths if 'h' in i.split('\\')[-1]]
sch_files_paths = [i for i in files_paths if 's' in i.split('\\')[-1]]

In [ ]:
%%capture
sample_data = read_data(healthy_files_paths[0])

In [ ]:
sample_data.shape #Number of epochs, number of channels, length of signal 

#EEG epoching is a procedure in which specific time-windows are extracted from the continuous EEG signal.
# These time windows are called “epochs”, and usually are time-locked with respect an event e.g. a visual stimulus 

## Convert the data into features and labelled arrays

In [ ]:
%%capture
healthy_epochs_arrays = [ read_data(i) for i in healthy_files_paths]
sch_epochs_arrays = [ read_data(i) for i in sch_files_paths]

In [ ]:
healthy_epochs_labels = [len(i)*[0] for i in healthy_epochs_arrays]
sch_epochs_labels = [len(i)*[1] for i in sch_epochs_arrays]

In [ ]:
#interleave the data so that each datapoint is alternated between one healthy and one patient, even is healthy and odd is patient
data_list = []
label_list = []

for i in range(len(sch_epochs_arrays)):
    data_list.append(healthy_epochs_arrays[i])
    data_list.append(sch_epochs_arrays[i])

    label_list.append(healthy_epochs_labels[i])
    label_list.append(sch_epochs_labels[i])

In [ ]:
# to avoid data leakage of multiple arrays from the same patient to be split into training and testing
# we will group the data so that its categorized for each patient which will will be considered in the splitting later
grouped_list = [[i]*len(j) for i,j in enumerate(data_list)]

In [ ]:
epochs=10

## ML CLassification

In [ ]:
data_array = np.vstack(data_list)
labels_array = np.hstack(label_list)
groups_array = np.hstack(grouped_list)
print(data_array.shape,labels_array.shape,groups_array.shape)

its 7201 rows because there are 28 patients (14 sch and 14 healthy), which average 257 epochs, so the whole data list would have 257*28

### Feature Extraction

each feature has shape 7201,19

In [ ]:
from scipy import stats
def mean(data):
    return np.mean(data,axis=-1)
    
def std(data):
    return np.std(data,axis=-1)

def ptp(data):
    return np.ptp(data,axis=-1) #maximum - minimum (range of values)

def var(data):
        return np.var(data,axis=-1)

def minim(data):
      return np.min(data,axis=-1)


def maxim(data):
      return np.max(data,axis=-1)

def argminim(data):
      return np.argmin(data,axis=-1)

def argmaxim(data):
      return np.argmax(data,axis=-1)

def mean_square(data):
      return np.mean(data**2,axis=-1)

def rms(data): #root mean square
    return  np.sqrt(np.mean(data**2,axis=-1))  

def abs_diffs_signal(data):
    return np.sum(np.abs(np.diff(data,axis=-1)),axis=-1)

def skewness(data):
    return stats.skew(data,axis=-1)

def kurtosis(data):
    return stats.kurtosis(data,axis=-1)

def concatenate_features(data):
    return np.concatenate((mean(data),std(data),ptp(data),var(data),minim(data),maxim(data),argminim(data),argmaxim(data),
                          mean_square(data),rms(data),abs_diffs_signal(data),
                          skewness(data),kurtosis(data)),axis=-1)

In [ ]:
features = []

for d in data_array:
    features.append(concatenate_features(d))

In [ ]:
features_array = np.array(features)
features_array.shape   # so we have 13 features per channel, and we have 19 channels, thats why its 7201x247

In [ ]:
from sklearn.linear_model import LogisticRegression
from sklearn.pipeline import Pipeline
from sklearn.preprocessing import StandardScaler
from sklearn.model_selection import GroupKFold,GridSearchCV,cross_val_score,cross_validate 

#### Logistic Regression

In [ ]:
clf=LogisticRegression(max_iter=1000)
gkf=GroupKFold(n_splits=5)
param_grid = {'classifier__C': [0.1,0.3,0.5,0.7,1,3]}
pipe=Pipeline([('scaler',StandardScaler()),('classifier',clf)])
gscv=GridSearchCV(pipe,param_grid,cv=gkf)
gscv.fit(features,labels_array,groups=groups_array)

In [ ]:
gscv.best_score_

#### Random Forest

In [ ]:
from sklearn.ensemble import RandomForestClassifier
from sklearn.preprocessing import StandardScaler
from sklearn.pipeline import make_pipeline

rdm=RandomForestClassifier()
gkf=GroupKFold(n_splits=5)
param_grid = {'classifier__n_estimators': [10,50,100,200]}
pipe=Pipeline([('scaler',StandardScaler()),('classifier',rdm)])
pipeline_rf=GridSearchCV(pipe,param_grid,cv=gkf)
# Random Forest pipeline
pipeline_rf.fit(features, labels_array, groups=groups_array)

In [ ]:
pipeline_rf.best_score_

## 1D-CNN

first, we check that we have the gpu running

In [ ]:
one_d_cnn_data_array=np.vstack(data_list)
one_d_cnn_label_array=np.hstack(label_list)
one_d_cnn_group_array=np.hstack(grouped_list)

print(one_d_cnn_data_array.shape,one_d_cnn_label_array.shape,one_d_cnn_group_array.shape)

In [ ]:
one_d_cnn_data_array=np.moveaxis(one_d_cnn_data_array,1,2) #cnn in keras expects the input channels at the end
print(one_d_cnn_data_array.shape)

In [ ]:
def cnnmodel():
    K.clear_session()
    model=Sequential()
    
    model.add(Conv1D(filters=10,kernel_size=8,strides=2,input_shape=(one_d_cnn_data_array.shape[1],one_d_cnn_data_array.shape[2])))#1
    model.add(BatchNormalization())
    model.add(LeakyReLU())
    model.add(AveragePooling1D(pool_size=2,strides=2))#2
    model.add(Dropout(0.35))

    model.add(Conv1D(filters=5,kernel_size=5,strides=1))#9
    model.add(LeakyReLU())
    model.add(AveragePooling1D(pool_size=3,strides=1))#2
    #model.add(GlobalAveragePooling1D())#10

    model.add(Flatten())

    
    model.add(Dense(128, activation='leaky_relu'))  # Additional Dense layer
    model.add(Dropout(0.4))  # Dropout to prevent overfitting
    model.add(Dense(32, activation='leaky_relu'))  # Additional Dense layer

    
    model.add(Dense(1,activation='sigmoid'))#11

    
    optimizer = Adam(learning_rate=0.0005)  # You can modify the learning rate here

    model.compile(optimizer=optimizer,loss='binary_crossentropy',metrics=['accuracy'])
    return model

model=cnnmodel()
model.summary()

#### Initial CNN

In [ ]:
def cnnmodel():
    clear_session()
    model=Sequential()
    
    model.add(Conv1D(filters=5,kernel_size=3,strides=1,input_shape=(one_d_cnn_data_array.shape[1],one_d_cnn_data_array.shape[2])))#1
    model.add(BatchNormalization())
    model.add(LeakyReLU())
    model.add(MaxPool1D(pool_size=2,strides=2))#2

    
    model.add(Conv1D(filters=5,kernel_size=3,strides=1))#3
    model.add(LeakyReLU())
    model.add(MaxPool1D(pool_size=2,strides=2))#4
    model.add(Dropout(0.5))

    model.add(Conv1D(filters=5,kernel_size=3,strides=1))#5
    model.add(LeakyReLU())
    model.add(AveragePooling1D(pool_size=2,strides=2))#6
    model.add(Dropout(0.5))
    
    model.add(Conv1D(filters=5,kernel_size=3,strides=1))#7
    model.add(LeakyReLU())
    model.add(AveragePooling1D(pool_size=2,strides=2))#8
    
    model.add(Conv1D(filters=5,kernel_size=3,strides=1))#9
    model.add(LeakyReLU())
    model.add(GlobalAveragePooling1D())#10
    
    model.add(Dense(1,activation='sigmoid'))#11
    
    model.compile('adam',loss='binary_crossentropy',metrics=['accuracy'])
    return model

model=cnnmodel()
model.summary()

In [ ]:
  # Importing Keras backend (TensorFlow) for clear_session()
gkf=GroupKFold()
# Initialize lists to store metrics for each fold
accuracies = []
training_losses = []
validation_losses = []
training_accuracies = []
validation_accuracies = []
confusion_matrices = []


for train_index, val_index in gkf.split(one_d_cnn_data_array, one_d_cnn_label_array, groups=one_d_cnn_group_array):
    train_features,train_labels=one_d_cnn_data_array[train_index],one_d_cnn_label_array[train_index]
    val_features,val_labels=one_d_cnn_data_array[val_index],one_d_cnn_label_array[val_index]
    
    scaler=StandardScaler()

    train_features = scaler.fit_transform(train_features.reshape(-1, train_features.shape[-1])).reshape(train_features.shape)
    val_features = scaler.transform(val_features.reshape(-1, val_features.shape[-1])).reshape(val_features.shape)
    
    K.clear_session()
    model=cnnmodel()
    
    history = model.fit(train_features,train_labels,
              epochs=10,
              batch_size=128,
              validation_data=(val_features,val_labels)
             )
    
    # Append loss and accuracy from the training history
    training_losses.append(history.history['loss'])
    validation_losses.append(history.history['val_loss'])
    training_accuracies.append(history.history['accuracy'])
    validation_accuracies.append(history.history['val_accuracy'])

    # Generate and store confusion matrix for the fold
    raw_predictions = model.predict(val_features)
    predictions = (raw_predictions > 0.5).astype(int)
    cm = confusion_matrix(val_labels, predictions)

    confusion_matrices.append(cm)

In [ ]:
mean_training_losses = np.mean(training_losses,axis=0)
mean_validation_losses = np.mean(validation_losses,axis=0)
mean_training_accuracies = np.mean(training_accuracies,axis=0)
mean_validation_accuracies = np.mean(validation_accuracies,axis=0)

In [ ]:
# Plotting
plt.figure(figsize=(10, 6))
plt.plot(mean_training_losses, label='Average Training Loss')
plt.plot(mean_validation_losses, label='Average Validation Loss')
plt.title('Training and Validation Loss Across All Folds')
plt.xlabel('Epoch')
plt.ylabel('Loss')
plt.legend()
plt.grid(True)
plt.show()

very clear overfitting

In [ ]:
# Plotting the confusion matrix for each fold
plt.figure(figsize=(10, 7))
sns.heatmap(cm, annot=True, fmt='d', cmap='Blues', xticklabels=['Healthy', 'Schizophrenic'], yticklabels=['Healthy', 'Schizophrenic'])
plt.title('Confusion Matrix')
plt.ylabel('True Label')
plt.xlabel('Predicted Label')
plt.show()

In [ ]:
#gkf=GroupKFold(n_splits=4)
logo = LeaveOneGroupOut()

K.clear_session()

# Initialize lists to store metrics for each fold
accuracies = []
training_losses = []
validation_losses = []
training_accuracies = []
validation_accuracies = []
confusion_matrices = []

#shuffled_groups = shuffle_group_labels(one_d_cnn_group_array)

for train_index, val_index in logo.split(one_d_cnn_data_array, one_d_cnn_label_array, groups=one_d_cnn_group_array):
    print("\n\n")

    print("Test group:", np.unique(one_d_cnn_group_array[val_index]))
    train_features,train_labels=one_d_cnn_data_array[train_index],one_d_cnn_label_array[train_index]
    val_features,val_labels=one_d_cnn_data_array[val_index],one_d_cnn_label_array[val_index]
    
    scaler=StandardScaler()

    train_features = scaler.fit_transform(train_features.reshape(-1, train_features.shape[-1])).reshape(train_features.shape)
    val_features = scaler.transform(val_features.reshape(-1, val_features.shape[-1])).reshape(val_features.shape)
    
    K.clear_session()
    model=cnnmodel()
    
    history = model.fit(train_features,train_labels,
              epochs=8,
              batch_size=256,
              validation_data=(val_features,val_labels)
             )
    
    # Append loss and accuracy from the training history
    training_losses.append(history.history['loss'])
    validation_losses.append(history.history['val_loss'])
    training_accuracies.append(history.history['accuracy'])
    validation_accuracies.append(history.history['val_accuracy'])


    # Generate and store confusion matrix for the fold
    raw_predictions = model.predict(val_features)
    predictions = (raw_predictions > 0.5).astype(int)
    cm = confusion_matrix(val_labels, predictions)

    confusion_matrices.append(cm)
    print(classification_report(val_labels, predictions))

    
    plt.figure(figsize=(10, 7))
    sns.heatmap(cm, annot=True, fmt='d', cmap='Blues', xticklabels=['Healthy', 'Schizophrenic'], yticklabels=['Healthy', 'Schizophrenic'])
    plt.title('Confusion Matrix')
    plt.ylabel('True Label')
    plt.xlabel('Predicted Label')
    plt.show()




mean_training_losses = np.mean(training_losses,axis=0)
mean_validation_losses = np.mean(validation_losses,axis=0)
mean_training_accuracies = np.mean(training_accuracies,axis=0)
mean_validation_accuracies = np.mean(validation_accuracies,axis=0)

# Plotting
plt.figure(figsize=(10, 6))
plt.plot(mean_training_losses, label='Average Training Loss')
plt.plot(mean_validation_losses, label='Average Validation Loss')
plt.title('Training and Validation Loss Across All Folds')
plt.xlabel('Epoch')
plt.ylabel('Loss')
plt.legend()
plt.grid(True)
plt.show()


Badly classified:

labels : 0,9,13,16,21,24

### BETTER CNN

We will change the way we are classifying patients based on a majority vote for each epoch (stimuli):

Explanation:

Predictions: Each epoch within a group is predicted, and then the mean of these predictions determines the group's classification.

Thresholding: If the mean prediction exceeds 0.5, the group is classified as schizophrenic; otherwise, it is classified as healthy.

Metrics Calculation: The predictions for each patient are used to update the confusion matrix and other metrics.

This approach evaluates the model based on its ability to classify patients correctly, not just individual epochs, aligning more closely with clinical diagnostic processes.

In [ ]:
model=cnnmodel()
model.summary()

#### LOGO

In [ ]:
best_loss = float('inf')
best_model_path = 'best_1D_CNN_model_LOGO.h5'

best_model_weights = None
best_fold_info = None


#gkf=GroupKFold(n_splits=4)
logo = LeaveOneGroupOut()

# Initialize lists to store metrics for each fold
accuracies = []
training_losses = []
validation_losses = []
training_accuracies = []
validation_accuracies = []
confusion_matrices = []

group_predictions = []
actual_group_labels = []
#shuffled_groups = shuffle_group_labels(one_d_cnn_group_array)

for train_index, val_index in logo.split(one_d_cnn_data_array, one_d_cnn_label_array, groups=one_d_cnn_group_array):
    print("\n\n")

    print("Test group:", np.unique(one_d_cnn_group_array[val_index]))
    train_features,train_labels=one_d_cnn_data_array[train_index],one_d_cnn_label_array[train_index]
    val_features,val_labels=one_d_cnn_data_array[val_index],one_d_cnn_label_array[val_index]

    
    scaler=StandardScaler()

    train_features = scaler.fit_transform(train_features.reshape(-1, train_features.shape[-1])).reshape(train_features.shape)
    val_features = scaler.transform(val_features.reshape(-1, val_features.shape[-1])).reshape(val_features.shape)
    
    #K.clear_session()
    model=cnnmodel()
    
    # Setup ReduceLROnPlateau callback
    #reduce_lr = ReduceLROnPlateau(monitor='val_accuracy', factor=0.1, patience=2, min_lr=0.00001, verbose=2)
    
    history = model.fit(train_features,train_labels,
              epochs=epochs,
              batch_size=128,
              shuffle=True,
              validation_data=(val_features,val_labels)
             )
    
    # Append loss and accuracy from the training history

    #print(history.history['val_loss'])
    training_losses.append(history.history['loss'])
    validation_losses.append(history.history['val_loss'])
    training_accuracies.append(history.history['accuracy'])
    validation_accuracies.append(history.history['val_accuracy'])

    raw_predictions = model.predict(val_features)
    
    group_prediction = np.mean(raw_predictions) > 0.6
    print("Averaged Prediction: :", np.mean(raw_predictions))
    #print("sample of predictions: :", raw_predictions[0:10])
    actual_group_label = np.mean(val_labels) > 0.5

    group_predictions.append(group_prediction)  # True if average prediction > 0.5, False otherwise
    actual_group_labels.append(actual_group_label)  # Assumes all labels in the group are the same

    accuracies.append(((group_prediction == actual_group_label)*1).astype(int))
    print("Correctly Identified: ",  group_prediction == actual_group_label)


    if history.history['val_loss'][-1] < best_loss:
        best_loss = history.history['val_loss'][-1]
        # Save the model
        model.save(best_model_path)
        print(f"Saved improved model with validation loss: {history.history['val_loss'][-1]}")

mean_training_losses = np.mean(training_losses,axis=0)
mean_validation_losses = np.mean(validation_losses,axis=0)
mean_training_accuracies = np.mean(training_accuracies,axis=0)
mean_validation_accuracies = np.mean(validation_accuracies,axis=0)

print("Final Validation Accuracy: ", np.mean(accuracies))
      
# Generate and store confusion matrix for the fold

final_cm = confusion_matrix(actual_group_labels, group_predictions)

plt.figure(figsize=(10, 7))
sns.heatmap(final_cm, annot=True, fmt='d', cmap='Blues', xticklabels=['Healthy', 'Schizophrenic'], yticklabels=['Healthy', 'Schizophrenic'])
plt.title('Confusion Matrix')
plt.ylabel('True Label')
plt.xlabel('Predicted Label')
plt.show()

print(classification_report(actual_group_labels, group_predictions))
ep = range(1,11)
# Plotting
plt.figure(figsize=(10, 6))
plt.plot(ep,mean_training_losses, label='Average Training Loss')
plt.plot(ep,mean_validation_losses,label='Average Validation Loss')
plt.title('Training and Validation Loss Across All Folds')
plt.xlabel('Epoch')
plt.ylabel('Loss')
plt.legend()
plt.grid(True)
plt.show()


In [ ]:
plot_training_results(training_losses,
                      training_accuracies,
                      validation_losses, 
                      validation_accuracies,
                      epochs=epochs,
                      save_plots=True,
                      save_dir='plots/5sec/1dcnn/logo')

#### GroupKFold

In [ ]:
epochs=10

gkf=GroupKFold(n_splits=5)
# Initialize lists to store metrics for each fold
accuracies = []
training_losses = []
validation_losses = []
training_accuracies = []
validation_accuracies = []



best_loss = float('inf')
best_model_path = 'best_1dcnn_model_GroupKFold.h5'

best_model_weights = None
best_fold_info = None

# Setup TensorBoard callback
log_dir = "logs/5sec/1dcnn/GroupKFold" + datetime.datetime.now().strftime("%Y%m%d-%H%M%S")
tensorboard_callback = TensorBoard(log_dir=log_dir, histogram_freq=1,update_freq='epoch')


shuffled_groups = shuffle_group_labels(one_d_cnn_group_array)


for train_index, val_index in gkf.split(one_d_cnn_data_array, one_d_cnn_label_array, groups=shuffled_groups):
    train_features,train_labels=one_d_cnn_data_array[train_index],one_d_cnn_label_array[train_index]
    val_features,val_labels=one_d_cnn_data_array[val_index],one_d_cnn_label_array[val_index]
    
    scaler=StandardScaler()

    train_features = scaler.fit_transform(train_features.reshape(-1, train_features.shape[-1])).reshape(train_features.shape)
    val_features = scaler.transform(val_features.reshape(-1, val_features.shape[-1])).reshape(val_features.shape)
    
    
    model=cnnmodel()
    
    history = model.fit(train_features,train_labels,
              epochs=epochs,
              initial_epoch=0,
              batch_size=128,
              shuffle=True,
              validation_data=(val_features,val_labels)
             )
    
    # Append loss and accuracy from the training history
    training_losses.append(history.history['loss'])
    validation_losses.append(history.history['val_loss'])
    training_accuracies.append(history.history['accuracy'])
    validation_accuracies.append(history.history['val_accuracy'])

    # Generate and store confusion matrix for the fold
    raw_predictions = model.predict(val_features)
    predictions = (raw_predictions > 0.5).astype(int)
    cm = confusion_matrix(val_labels, predictions)


    if history.history['val_loss'][-1] < best_loss:
        best_loss = history.history['val_loss'][-1]
        # Save the model
        model.save(best_model_path)
        print(f"Saved improved model with validation loss: {history.history['val_loss'][-1]}")

    print("Final Validation Accuracy: ", np.mean(validation_accuracies))
    # Plotting the confusion matrix for each fold
    plt.figure(figsize=(10, 7))
    sns.heatmap(cm, annot=True, fmt='d', cmap='Blues', xticklabels=['Healthy', 'Schizophrenic'], yticklabels=['Healthy', 'Schizophrenic'])
    plt.title('Confusion Matrix')
    plt.ylabel('True Label')
    plt.xlabel('Predicted Label')
    plt.show()

In [ ]:
plot_training_results(training_losses,
                      training_accuracies,
                      validation_losses, 
                      validation_accuracies,
                      epochs=epochs,
                      save_plots=True,
                      save_dir='plots/5sec/1dcnn/gf')

## ChronoNet

In [ ]:
chrono_data_array=np.vstack(data_list)
chrono_label_array=np.hstack(label_list)
chrono_group_array=np.hstack(grouped_list)
chrono_data_array=np.moveaxis(chrono_data_array,1,2) #cnn in keras expects the input channels at the end

print(chrono_data_array.shape,chrono_label_array.shape,chrono_group_array.shape)

In [ ]:
def block(input):
  conv1 = Conv1D(32, 2, strides=2,activation='relu',padding="same")(input)
  conv2 = Conv1D(32, 4, strides=2,activation='relu',padding="causal")(input)
  conv3 = Conv1D(32, 8, strides=2,activation='relu',padding="causal")(input)
  x = concatenate([conv1,conv2,conv3],axis=2)
  return x


def ChronoNet():
    input= Input(shape=(1250,19))
    block1=block(input)
    block2=block(block1)
    block3=block(block2)
    
    #conv_o = AveragePooling1D(pool_size=2,strides=2)(block3)
    conv_o2 = BatchNormalization()(block3)
    gru_out1 = GRU(32,activation='tanh',return_sequences=True)(conv_o2)
    gru_out2 = GRU(32,activation='tanh',return_sequences=True)(gru_out1)
    gru_out = concatenate([gru_out1,gru_out2],axis=2)
    gru_out3 = GRU(32,activation='tanh',return_sequences=True)(gru_out)
    gru_out = concatenate([gru_out1,gru_out2,gru_out3])
    gru_out4 = GRU(32,activation='tanh')(gru_out)
    
    predictions = Dense(1,activation='sigmoid')(gru_out4)
    model = Model(inputs=input, outputs=predictions)
    optimizer = Adam(learning_rate=0.001)  # You can modify the learning rate here
    
    model.compile(optimizer = optimizer, loss = 'binary_crossentropy', metrics=['accuracy'])

    return model

model = ChronoNet()
model.summary()

#### LOGO Validation

In [ ]:
best_loss = float('inf')
best_model_path = 'best_5s_chrono_model_LOGO.h5'

best_model_weights = None
best_fold_info = None


logo = LeaveOneGroupOut()

# Initialize lists to store metrics for each fold
accuracies = []
training_losses = []
validation_losses = []
training_accuracies = []
validation_accuracies = []
confusion_matrices = []

group_predictions = []
actual_group_labels = []
#shuffled_groups = shuffle_group_labels(one_d_cnn_group_array)


# Setup TensorBoard callback
log_dir = "logs/duration/chrono/LOGO" + datetime.datetime.now().strftime("%Y%m%d-%H%M%S")
tensorboard_callback = TensorBoard(log_dir=log_dir, histogram_freq=1)

shuffled_groups = shuffle_group_labels(chrono_group_array)
for train_index, val_index in logo.split(chrono_data_array, chrono_label_array, groups=chrono_group_array):
    print("\n\n")

    print("Test group:", np.unique(chrono_group_array[val_index]))
    train_features,train_labels=chrono_data_array[train_index],chrono_label_array[train_index]
    val_features,val_labels=chrono_data_array[val_index],chrono_label_array[val_index]

    
    scaler=StandardScaler()

    train_features = scaler.fit_transform(train_features.reshape(-1, train_features.shape[-1])).reshape(train_features.shape)
    val_features = scaler.transform(val_features.reshape(-1, val_features.shape[-1])).reshape(val_features.shape)
    
    model=ChronoNet()
    
    history = model.fit(train_features,train_labels,
              epochs=epochs,
              batch_size=128,
              shuffle=True,
              validation_data=(val_features,val_labels)
             )
    
    # Append loss and accuracy from the training history

    #print(history.history['val_loss'])
    training_losses.append(history.history['loss'])
    validation_losses.append(history.history['val_loss'])
    training_accuracies.append(history.history['accuracy'])
    validation_accuracies.append(history.history['val_accuracy'])
    
    raw_predictions = model.predict(val_features)
    
    group_prediction = np.mean(raw_predictions) > 0.5
    print("Averaged Prediction: :", np.mean(raw_predictions))
    actual_group_label = np.mean(val_labels) > 0.5

    group_predictions.append(group_prediction)  # True if average prediction > 0.5, False otherwise
    actual_group_labels.append(actual_group_label)  # Assumes all labels in the group are the same

    accuracies.append(((group_prediction == actual_group_label)*1).astype(int))
    print("Correctly Identified: ",  group_prediction == actual_group_label)


    if history.history['val_loss'][-1] < best_loss:
        best_loss = history.history['val_loss'][-1]
        # Save the model
        model.save(best_model_path)
        print(f"Saved improved model with validation loss: {history.history['val_loss'][-1]}")

mean_training_losses = np.mean(training_losses,axis=0)
mean_validation_losses = np.mean(validation_losses,axis=0)
mean_training_accuracies = np.mean(training_accuracies,axis=0)
mean_validation_accuracies = np.mean(validation_accuracies,axis=0)

print("Final Validation Accuracy: ", np.mean(accuracies))
      
# Generate and store confusion matrix for the fold

final_cm = confusion_matrix(actual_group_labels, group_predictions)

plt.figure(figsize=(10, 7))
sns.heatmap(final_cm, annot=True, fmt='d', cmap='Blues', xticklabels=['Healthy', 'Schizophrenic'], yticklabels=['Healthy', 'Schizophrenic'])
plt.title('Confusion Matrix')
plt.ylabel('True Label')
plt.xlabel('Predicted Label')
plt.show()

print(classification_report(actual_group_labels, group_predictions))
ep = range(1,11)
# Plotting
plt.figure(figsize=(10, 6))
plt.plot(ep,mean_training_losses, label='Average Training Loss')
plt.plot(ep,mean_validation_losses,label='Average Validation Loss')
plt.title('Training and Validation Loss Across All Folds')
plt.xlabel('Epoch')
plt.ylabel('Loss')
plt.legend()
plt.grid(True)
plt.show()


In [ ]:
plot_training_results(training_losses,
                      training_accuracies,
                      validation_losses, 
                      validation_accuracies,
                      epochs=epochs,
                      save_plots=True,
                      save_dir='plots/5sec/chrono/logo',
                      leg = False)

#### GroupKFold Validation

In [ ]:
gkf=GroupKFold(n_splits=5)

# Initialize lists to store metrics for each fold
accuracies = []
training_losses = []
validation_losses = []
training_accuracies = []
validation_accuracies = []


best_loss = float('inf')
best_model_path = 'best_chrono_model_GroupKFold.h5'

best_model_weights = None
best_fold_info = None

# Setup TensorBoard callback
log_dir = "logs/duration/chrono/GroupKFold" + datetime.datetime.now().strftime("%Y%m%d-%H%M%S")
tensorboard_callback = TensorBoard(log_dir=log_dir, histogram_freq=1)

shuffled_groups = shuffle_group_labels(chrono_group_array)

for train_index, val_index in gkf.split(chrono_data_array, chrono_label_array, groups=shuffled_groups):
    train_features,train_labels=chrono_data_array[train_index],chrono_label_array[train_index]
    val_features,val_labels=chrono_data_array[val_index],chrono_label_array[val_index]
    
    scaler=StandardScaler()

    train_features = scaler.fit_transform(train_features.reshape(-1, train_features.shape[-1])).reshape(train_features.shape)
    val_features = scaler.transform(val_features.reshape(-1, val_features.shape[-1])).reshape(val_features.shape)

    model=ChronoNet()
    
    history = model.fit(train_features,train_labels,
              epochs=10,
              batch_size=128,
              shuffle=True,
              validation_data=(val_features,val_labels)
             )
    
    # Append loss and accuracy from the training history
    training_losses.append(history.history['loss'])
    validation_losses.append(history.history['val_loss'])
    training_accuracies.append(history.history['accuracy'])
    validation_accuracies.append(history.history['val_accuracy'])

    # Generate and store confusion matrix for the fold
    raw_predictions = model.predict(val_features)
    predictions = (raw_predictions > 0.5).astype(int)
    cm = confusion_matrix(val_labels, predictions)


    if history.history['val_loss'][-1] < best_loss:
        best_loss = history.history['val_loss'][-1]
        # Save the model
        model.save(best_model_path)
        print(f"Saved improved model with validation loss: {history.history['val_loss'][-1]}")

    print("Final Validation Accuracy: ", np.mean(validation_accuracies,axis=0))
    # Plotting the confusion matrix for each fold
    plt.figure(figsize=(10, 7))
    sns.heatmap(cm, annot=True, fmt='d', cmap='Blues', xticklabels=['Healthy', 'Schizophrenic'], yticklabels=['Healthy', 'Schizophrenic'])
    plt.title('Confusion Matrix')
    plt.ylabel('True Label')
    plt.xlabel('Predicted Label')
    plt.show()

In [ ]:
plot_training_results(training_losses,
                      training_accuracies,
                      validation_losses, 
                      validation_accuracies,
                      epochs=epochs,
                      save_plots=True,
                      save_dir='plots/5sec/chrono/gf')

## Custom CNN-GRU Model

In [ ]:
def gru_cnn_model():
    model = Sequential()
    
    # Convolutional blocks
    model.add(Conv1D(filters=32, kernel_size=2, strides=2, activation='relu', padding="same", input_shape=(chrono_data_array.shape[1], chrono_data_array.shape[2])))
    model.add(Conv1D(filters=32, kernel_size=4, strides=2, activation='relu', padding="causal"))
    model.add(Conv1D(filters=32, kernel_size=8, strides=2, activation='relu', padding="causal"))
    model.add(BatchNormalization())
    
    # Merging GRU operations in a way that fits sequential model constraints
    model.add(GRU(32, return_sequences=True, activation='tanh'))
    model.add(GRU(32, return_sequences=True, activation='tanh'))
    
    # Let's simplify by assuming a final GRU handles the combined effect
    model.add(GRU(32, return_sequences=False, activation='tanh'))  # Final GRU reduces to single vector

    # Output layer
    model.add(Dense(1, activation='sigmoid'))

    optimizer = Adam(learning_rate=0.0005) 

    model.compile(optimizer=optimizer, loss='binary_crossentropy', metrics=['accuracy'])
    
    return model

# Create and compile the model
model = gru_cnn_model()
model.summary()

In [ ]:
def gru_cnn_model2(input_shape):
    inputs = Input(shape=input_shape)

    # Convolutional blocks
    x1 = Conv1D(filters=32, kernel_size=2, strides=2, activation='relu', padding="same")(inputs)
    x2 = Conv1D(filters=32, kernel_size=4, strides=2, activation='relu', padding="causal")(inputs)
    x3 = Conv1D(filters=32, kernel_size=8, strides=2, activation='relu', padding="causal")(inputs)
    x = Concatenate()([x1, x2, x3])
    x = BatchNormalization()(x)

    # GRU blocks with concatenation
    gru1 = GRU(32, return_sequences=True, activation='tanh')(x)
    gru2 = GRU(32, return_sequences=True, activation='tanh')(gru1)
    gru_concat1 = Concatenate(axis=-1)([gru1, gru2])
    #print(gru_concat1.shape)
    gru3 = GRU(32, return_sequences=True, activation='tanh')(gru_concat1)
    #print(gru3.shape)

    gru_concat2 = Concatenate(axis=-1)([gru1, gru2, gru3])
    #print(gru_concat2.shape)

    gru4 = GRU(32, return_sequences=False, activation='tanh')(gru_concat2)
    #print(gru4.shape)

    # Output layer
    outputs = Dense(1, activation='sigmoid')(gru4)

    model = Model(inputs=inputs, outputs=outputs)

    optimizer = Adam(learning_rate=0.0005)
    model.compile(optimizer=optimizer, loss='binary_crossentropy', metrics=['accuracy'])
    
    return model

# Assuming chrono_data_array.shape = (batch_size, sequence_length, num_features)
model = ChronoNet(input_shape=(chrono_data_array.shape[1], chrono_data_array.shape[2]))
model.summary()

#### LOGO

In [ ]:
best_loss = float('inf')
best_model_path = 'best_5s_customcnn_gru_model_LOGO.h5'

best_model_weights = None
best_fold_info = None


logo = LeaveOneGroupOut()

# Initialize lists to store metrics for each fold
accuracies = []
training_losses = []
validation_losses = []
training_accuracies = []
validation_accuracies = []
confusion_matrices = []

group_predictions = []
actual_group_labels = []
#shuffled_groups = shuffle_group_labels(one_d_cnn_group_array)


# Setup TensorBoard callback
log_dir = "logs/duration/custom/LOGO" + datetime.datetime.now().strftime("%Y%m%d-%H%M%S")
tensorboard_callback = TensorBoard(log_dir=log_dir, histogram_freq=1)

#shuffled_groups = shuffle_group_labels(chrono_group_array)
for train_index, val_index in logo.split(chrono_data_array, chrono_label_array, groups=chrono_group_array):
    print("\n\n")

    print("Test group:", np.unique(chrono_group_array[val_index]))
    train_features,train_labels=chrono_data_array[train_index],chrono_label_array[train_index]
    val_features,val_labels=chrono_data_array[val_index],chrono_label_array[val_index]

    
    scaler=StandardScaler()

    train_features = scaler.fit_transform(train_features.reshape(-1, train_features.shape[-1])).reshape(train_features.shape)
    val_features = scaler.transform(val_features.reshape(-1, val_features.shape[-1])).reshape(val_features.shape)
    
    model=gru_cnn_model()
    
    history = model.fit(train_features,train_labels,
              epochs=10,
              batch_size=128,
              shuffle=True,
              validation_data=(val_features,val_labels)
             )
    
    # Append loss and accuracy from the training history

    #print(history.history['val_loss'])
    training_losses.append(history.history['loss'])
    validation_losses.append(history.history['val_loss'])
    training_accuracies.append(history.history['accuracy'])
    validation_accuracies.append(history.history['val_accuracy'])
    
    raw_predictions = model.predict(val_features)
    
    group_prediction = np.mean(raw_predictions) > 0.5
    print("Averaged Prediction: :", np.mean(raw_predictions))
    actual_group_label = np.mean(val_labels) > 0.5

    group_predictions.append(group_prediction)  # True if average prediction > 0.5, False otherwise
    actual_group_labels.append(actual_group_label)  # Assumes all labels in the group are the same

    accuracies.append(((group_prediction == actual_group_label)*1).astype(int))
    print("Correctly Identified: ",  group_prediction == actual_group_label)


    if history.history['val_loss'][-1] < best_loss:
        best_loss = history.history['val_loss'][-1]
        # Save the model
        #model.save(best_model_path)
        #print(f"Saved improved model with validation loss: {history.history['val_loss'][-1]}")

mean_training_losses = np.mean(training_losses,axis=0)
mean_validation_losses = np.mean(validation_losses,axis=0)
mean_training_accuracies = np.mean(training_accuracies,axis=0)
mean_validation_accuracies = np.mean(validation_accuracies,axis=0)

print("Final Validation Accuracy: ", np.mean(accuracies))
      
# Generate and store confusion matrix for the fold

final_cm = confusion_matrix(actual_group_labels, group_predictions)

plt.figure(figsize=(10, 7))
sns.heatmap(final_cm, annot=True, fmt='d', cmap='Blues', xticklabels=['Healthy', 'Schizophrenic'], yticklabels=['Healthy', 'Schizophrenic'])
plt.title('Confusion Matrix')
plt.ylabel('True Label')
plt.xlabel('Predicted Label')
plt.show()

print(classification_report(actual_group_labels, group_predictions))
ep = range(1,6)
# Plotting
plt.figure(figsize=(10, 6))
plt.plot(ep,mean_training_losses, label='Average Training Loss')
plt.plot(ep,mean_validation_losses,label='Average Validation Loss')
plt.title('Training and Validation Loss Across All Folds')
plt.xlabel('Epoch')
plt.ylabel('Loss')
plt.legend()
plt.grid(True)
plt.show()


In [ ]:
plot_training_results(training_losses,
                      training_accuracies,
                      validation_losses, 
                      validation_accuracies,
                      epochs=epochs,
                      save_plots=True,
                      save_dir='plots/5sec/customcnn_gru/logo')

#### GroupKFold

In [ ]:
gkf=GroupKFold(n_splits=5)

# Initialize lists to store metrics for each fold
accuracies = []
training_losses = []
validation_losses = []
training_accuracies = []
validation_accuracies = []


best_loss = float('inf')
best_model_path = 'best_customcnn_gru_GroupKFold.h5'

best_model_weights = None
best_fold_info = None

# Setup TensorBoard callback
log_dir = "logs/duration/custom/GroupKFold" + datetime.datetime.now().strftime("%Y%m%d-%H%M%S")
tensorboard_callback = TensorBoard(log_dir=log_dir, histogram_freq=1)

shuffled_groups = shuffle_group_labels(chrono_group_array)

for train_index, val_index in gkf.split(chrono_data_array, chrono_label_array, groups=shuffled_groups):
    train_features,train_labels=chrono_data_array[train_index],chrono_label_array[train_index]
    val_features,val_labels=chrono_data_array[val_index],chrono_label_array[val_index]
    
    scaler=StandardScaler()

    train_features = scaler.fit_transform(train_features.reshape(-1, train_features.shape[-1])).reshape(train_features.shape)
    val_features = scaler.transform(val_features.reshape(-1, val_features.shape[-1])).reshape(val_features.shape)
    
    K.clear_session()
    model=gru_cnn_model()
    
    history = model.fit(train_features,train_labels,
              epochs=10,
              batch_size=128,
              shuffle=True,
              validation_data=(val_features,val_labels)
             )
    
    # Append loss and accuracy from the training history
    training_losses.append(history.history['loss'])
    validation_losses.append(history.history['val_loss'])
    training_accuracies.append(history.history['accuracy'])
    validation_accuracies.append(history.history['val_accuracy'])

    # Generate and store confusion matrix for the fold
    raw_predictions = model.predict(val_features)
    predictions = (raw_predictions > 0.5).astype(int)
    cm = confusion_matrix(val_labels, predictions)


    if history.history['val_loss'][-1] < best_loss:
        best_loss = history.history['val_loss'][-1]
        # Save the model
        model.save(best_model_path)
        print(f"Saved improved model with validation loss: {history.history['val_loss'][-1]}")

    print("Final Validation Accuracy: ", np.mean(validation_accuracies,axis=0))
    # Plotting the confusion matrix for each fold
    plt.figure(figsize=(10, 7))
    sns.heatmap(cm, annot=True, fmt='d', cmap='Blues', xticklabels=['Healthy', 'Schizophrenic'], yticklabels=['Healthy', 'Schizophrenic'])
    plt.title('Confusion Matrix')
    plt.ylabel('True Label')
    plt.xlabel('Predicted Label')
    plt.show()

In [ ]:
plot_training_results(training_losses,
                      training_accuracies,
                      validation_losses, 
                      validation_accuracies,
                      epochs=epochs,
                      save_plots=True,
                      save_dir='plots/5sec/customcnn_gru/gf')